In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import glob
import yaml

def parse_results_file(file_path):
    metrics = {}
    with open(file_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                try:
                    metrics[key.strip()] = float(value.strip())
                except ValueError:
                    metrics[key.strip()] = value.strip()
    return metrics
def get_cont_weight(yaml_path):
    with open(yaml_path, 'r') as file:
        config = yaml.safe_load(file)
    try:
        return config['CONT']['WEIGHTS']
    except KeyError as e:
        raise KeyError(f"Missing key in YAML file: {e}")

def main(exp_name, only_df = False):
    output_dir = exp_name
    os.makedirs(output_dir, exist_ok=True)

    step0_weight = get_cont_weight(os.path.join(exp_name, "step1/config.yaml"))
    root = exp_name.split(os.sep)[:-4]
    if len(root)>0:
        step0_path = os.path.dirname(os.path.join('/', *root, step0_weight))
    else:
        step0_path = os.path.dirname(os.path.join(*root, step0_weight))
    step_dirs = sorted(glob.glob(os.path.join(exp_name, 'step*')), 
                       key=lambda x: int(re.search(r'step(\d+)', x).group(1)))
    step_dirs = [step0_path] + step_dirs if 'step0' not in step_dirs[0] else step_dirs
    all_data = []
    step_names = []
    # 收集所有数据
    for step_dir in step_dirs:
        step_name = os.path.basename(step_dir)
        step_num = int(re.search(r'step(\d+)', step_name).group(1))
        file_path = os.path.join(step_dir, 'eval_results_all.txt')
        if os.path.exists(file_path):
            metrics = parse_results_file(file_path)
            metrics['Step'] = step_num
            all_data.append(metrics)
            step_names.append(step_name)
        else:
            print(f"{file_path} not exists")
            
    if not all_data:
        print("未找到有效数据")
        return
    
    df = pd.DataFrame(all_data)
    df = df.set_index('Step').sort_index()
    if only_df:
        return df
    plt.rcParams.update({
        'font.family': 'DejaVu Sans',
        'font.size': 10,
        'axes.titlesize': 14,
        'figure.dpi': 150,
        'savefig.bbox': 'tight',
        'savefig.pad_inches': 0.05,
        'figure.autolayout': True,
        'axes.titlepad': 10, 
    })
    
    cmap = LinearSegmentedColormap.from_list('custom_cmap', ['#f0f9e8', '#7bccc4', '#0868ac'])
    
    # ===== 1. 通用指标表格 - 行内正则化 =====
    general_metrics = ['AP', 'AP50', 'AP75', 'APs', 'APm', 'APl', 'AR1', 'AR10']
    general_df = df[general_metrics].T.round(2)
    fig_width = max(8, len(general_df.columns) * 1.5)
    fig_height = max(4, len(general_df.index) * 0.6) + 0.6
    
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.axis('off')
    ax.set_title('General Evaluation Metrics (Row-Normalized)', 
                 fontweight='bold', pad=6)
    
    table = ax.table(
        cellText=general_df.values,
        rowLabels=general_df.index,
        colLabels=[f'Step {i}' for i in general_df.columns],
        cellLoc='center',
        loc='center'
    )
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)

    for i in range(general_df.shape[0]):
        row_values = general_df.iloc[i, :].values
        row_min = np.min(row_values)
        row_max = np.max(row_values)
        
        for j in range(general_df.shape[1]):
            value = general_df.iloc[i, j]
            if row_max - row_min > 1e-6:
                normalized_value = (value - row_min) / (row_max - row_min)
            else:
                normalized_value = 0.5
            cell = table[i+1, j]
            cell.set_facecolor(cmap(normalized_value))
    

    plt.tight_layout(pad=0.1, rect=[0, 0, 1, 0.98]) 
    plt.savefig(os.path.join(output_dir, 'general_metrics_table.jpg'), 
                bbox_inches='tight', pad_inches=0.03)
    plt.close()
    
    # ===== 2. 类别AP表格 - 行内正则化 =====
    ap_columns = [col for col in df.columns if col.startswith('AP-')]
    ap_df = df[ap_columns].T.round(2)
    
    fig_width = max(10, len(ap_df.columns) * 1.5)
    fig_height = min(25, max(6, len(ap_df.index) * 0.4)) + 0.6
    
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.axis('off')
    ax.set_title('Per-Category AP Metrics (Row-Normalized)', 
                 fontweight='bold', pad=6)
    
    table = ax.table(
        cellText=ap_df.values,
        rowLabels=ap_df.index.str.replace('AP-', ''),
        colLabels=[f'Step {i}' for i in ap_df.columns],
        cellLoc='center',
        loc='center'
    )
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.6)
    
    for i in range(ap_df.shape[0]):
        row_values = ap_df.iloc[i, :].values

        non_zero_values = [v for v in row_values if v != 0]
        
        if non_zero_values:
            row_min = np.min(non_zero_values)
            row_max = np.max(non_zero_values)
        else:
            row_min, row_max = 0, 1
            
        for j in range(ap_df.shape[1]):
            value = ap_df.iloc[i, j]
            
            if value == 0:
                cell = table[i+1, j]
                cell.set_facecolor('#f0f0f0')
            else:
                if row_max - row_min > 1e-6:
                    normalized_value = (value - row_min) / (row_max - row_min)
                else:
                    normalized_value = 0.5
                
                cell = table[i+1, j]
                cell.set_facecolor(cmap(normalized_value))

    plt.tight_layout(pad=0.1, rect=[0, 0, 1, 0.98])
    plt.savefig(os.path.join(output_dir, 'category_ap_table.jpg'), 
                bbox_inches='tight', pad_inches=0.03)
    plt.close()
    
    print(f"表格已保存至: {output_dir}/general_metrics_table.jpg 和 {output_dir}/category_ap_table.jpg")
    return df
experiment_name = "continual_part_experiment_folder"
df = main(experiment_name)
data = df.to_numpy()[:, -40:]
n_task = data.shape[0]
res = []
c_base, c_inc = (20, 4)
eps = 1e-10
for t in range(n_task-1):
    s = c_base + c_inc * (t-1) if t > 0 else 0
    e = s + c_inc if t > 0 else c_base
    A = []
    for c in range(s, e):
        a = n_task - (t+1)
        if data[t, c] - data[-1 , c] >= 0:
            I = 1
        else:
            I = 0
        x = I * ((data[t, c] - data[-1, c]) / (data[t, c] + eps))
        y = x / a
        A.append(y)
    res.append(sum(A))
FR = sum(res)/40
print(FR)
for c in range(40):
    for i in range(n_task - 1):
        j = i + 1
        if data[i, c] - data[j , c] >= 0:
            a = 1
        else:
            a = 0
        res.append(a * (data[i, c] - data[j , c]))
y = sum(res)/len(res)
print(f"forgetting = {y}")

表格已保存至: results/exp_0701/ytvis_2019-vis_20-4-ov/ytvis_2019_inc20-4_split_ip_fx3_0811/general_metrics_table.jpg 和 results/exp_0701/ytvis_2019-vis_20-4-ov/ytvis_2019_inc20-4_split_ip_fx3_0811/category_ap_table.jpg
0.02111073726645005
forgetting = 0.7875686788755365
